# CSC416 - DigitalImage - HW#2 - Rinty Chowdhury

In [ ]:
import numpy as np
import math
from subtract import Subtract

## Helper functions

In [ ]:
def read_image_file(raw_image):
    pixels = [[0 for row in range(512)] for column in range(512)]
    with open(raw_image, 'rb') as binary_file:
        data = binary_file.read()
        string_data = " ".join(map(str, data))
        list_data = string_data.split(" ")
        counter = 0
        for i in range(512):
            for j in range(512):
                pixels[i][j] = list_data[counter]
                counter = counter + 1
    return pixels

def read_image_file2(image, n, dimension):
    t = [[0 for row in range(dimension[0])] for column in range(dimension[1])]
    with open(image, "r+b") as binary_file:
        data = binary_file.read_image_file()
        string_data = " ".join(map(str, data))
        list_data = string_data.split(" ")
        counter = 0
        for i in range(dimension[0] - n):
            for j in range(dimension[1] - n):
                t[i][j] = list_data[counter]
                counter = counter + 1
    return t
    
def move_image(image, move, dimension):
    result = [[0 for row in range(dimension[0])] for column in range(dimension[1])]
    for i in range(dimension[0]):
        for j in range(dimension[1]):
            if (((i + move + 1) <= 512) and ((j + move + 1) <= 512)):
                result[i + move][j + move] = image[i][j]
    return result

def out_of_range_pixel(image):
    min = 10000
    max = -1000
    for i in range(512):
        for j in range(512):
            if (int(image[i][j] > max)):
                max = image[i][j]
            if (int(image[i][j]) < min):
                min = image[i][j]
                
def image_normalization(image):
    result = [[0 for row in range(512)] for column in range(512)]
    for i in range(512):
        for j in range(512):
            result[i][j] = image[i][j] + 255
    return result

def write_image_file(pixels, image_file):
    new_file = open(image_file + '.raw', 'wb')
    formatted_pixels = np.matrix(pixels).astype('uint8')
    new_file.write(formatted_pixels)
    new_file.close()

In [ ]:
lena = "image_file\\lena.raw"
image = read_image_file(lena)

## Histogram processing

In [ ]:
def histogram_equalization(input_image, output_image):
    intensities = [0 for row in range(255)]
    probability = [0 for row in range(255)]
    cumulative_probability = [0 for row in range(255)]
    result = [[0 for row in range(512)] for column in range(512)]
    for i in range(512):
        for j in range(512):
            x = int(image[i][j])
            intensities[x] = intensities[x] + 1
    for i in range(255):
        intensity = int(intensities[i])
        probability[i] = float(intensity / ((512 * 512) - 1))
    for i in range(255):
        cumulative_probability[i] = cumulative_probability[i - 1] + probability[i]
    for i in range(512):
        for j in range(512):
            pixel_intensity = int(image[i][j])
            result[i][j] = int(float(cumulative_probability[pixel_intensity]) * 255)
    return (write_image_file(result, output_image))

histogram_equalization(lena, "lena_histequal")

## Gaussian filtering

In [ ]:
def gaussian_filter(input_image, kernel_size, sigma, k, output_image):
    new_dimension = [522, 522]
    image = read_image_file2(input_image, 10, new_dimension)
    new_image = move_image(image, 5, new_dimension)
    result = [[0 for row in range(512)] for column in range(512)]
    sigma = sigma
    k = k
    kernel = [[0 for row in range(kernel_size[0])] for column in range(kernel_size[1])]
    for i in range(5):
        for j in range(5):
            kernel[i][j] = K * math.exp(-(math.pow(i - 2, 2) + math.pow(j - 2, 2)) / (2 * math.pow(sigma, 2)))
    for i in range(512):
        for j in range(512):
            temp = 0
            for x in range(5):
                for y in range(5):
                    temp += float(new_image[i + x][j + y]) * kernel[x][y]
            result[i][j] = int(temp / 25)
    return (write_image_file(result, output_image))

gaussian_filter(lena, [5, 5], 2, 1.67, "lena_smooth")

## Unsharp masking

In [ ]:
def unsharp_mask(input_image, smooth_image, output_image):
    result = Subtract.sub_two_matrices(input_image, smooth_image, [512, 512])
    out_of_range_pixel(result)
    result = Subtract.add_two_matrices(input_image, result, [512, 512])
    min = 10000
    max = -1000
    for i in range(512):
        for j in range(512):
            if int(result[i][j]) < min:
                min = int(result[i][j])
    for i in range(512):
        for j in range(512):
            result[i][j] = result[i][j] - min
    for i in range(512):
        for j in range(512):
            if int(result[i][j]) > max:
                max = int(result[i][j])
    for i in range(512):
        for j in range(512):
            result[i][j] = (result[i][j] / max) 
    for i in range(512):
        for j in range(512):
            result[i][j] = int(result[i][j] * 255)
    out_of_range_pixel(result)
    return (write_image_file(result, output_image))
    
unsharp_mask(image, smooth, "lena_unsharpmask")

## Highpass filtering

In [ ]:
def highpass_filter(input_image, output_image):
    new_dimension = [518, 518]
    image = read_image_file2(input_image, 6, new_dimension)
    new_image = move_image(image, 5, new_dimension)
    result = [[0 for row in range(512)] for column in range(512)]
    laplacian_kernel = [[0, 1, 0], [1, -4, 1], [0, 1, 0]]
    for i in range(512):
        for j in range(512):
            temp = 0
            for x in range(3):
                for y in range(3):
                    temp += float(new_image[i + 3 + x - 1][j + 3 + y - 1]) * laplacian_kernel[x][y]
            result[i][j] = int(temp / 9)         
    result = Subtract.sub_two_matrices(image, result, [512, 512])
    result = image_normalization(result)
    return (write_image_file(result, output_image))

highpass_filter(lena, "lena_sharpening")